<a href="https://colab.research.google.com/github/SohaHussain/HuggingFace-course/blob/main/pipeline_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Transformers and Datasets library

In [1]:
!pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 306 kB 8.5 MB/s 
     |████████████████████████████████| 3.4 MB 54.3 MB/s 
     |████████████████████████████████| 243 kB 60.7 MB/s 
     |████████████████████████████████| 1.1 MB 49.3 MB/s 
     |████████████████████████████████| 133 kB 64.8 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 160 kB 62.0 MB/s 
     |████████████████████████████████| 271 kB 65.9 MB/s 
     |████████████████████████████████| 192 kB 60.8 MB/s 
     |████████████████████████████████| 596 kB 62.0 MB/s 
     |████████████████████████████████| 3.3 MB 41.9 MB/s 
     |████████████████████████████████| 895 kB 56.0 MB/s 
     |████████████████████████████████| 1.2 MB 55.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## using sentiment analysis pipeline

In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(["I have been waiting for this opportunity my whole life!","I hate it so much."])


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9987143278121948},
 {'label': 'NEGATIVE', 'score': 0.9995924830436707}]

#### The pipeline function groups three steps together.
1. pre-processing
2. going through the model
3. post-processing

## Pre-processing

Like other neural networks, Transformer models can’t process raw text directly, so the first step of our pipeline is to convert the text inputs into numbers that the model can make sense of. To do this we use a tokenizer, which will be responsible for:

Splitting the input into words, subwords, or symbols (like punctuation) that are called tokens
Mapping each token to an integer
Adding additional inputs that may be useful to the model
All this preprocessing needs to be done in exactly the same way as when the model was pretrained, so we first need to download that information from the Model Hub. To do this, we use the AutoTokenizer class and its from_pretrained() method. Using the checkpoint name of our model, it will automatically fetch the data associated with the model’s tokenizer and cache it.

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Once we have the tokenizer, we can directly pass our sentences to it and we’ll get back a dictionary that’s ready to feed to our model! The next thing we have to do is to convert the list of input IDs to tensors.

Transformer models only accept tensors as input. 
*return_tensors* argument is used to specify the type of tensors(PyTorch,TensorFlow, Numpy) we want to get back.

In [4]:
raw_input = ["I have been waiting for this opportunity my whole life!","I hate it so much."]

inputs = tokenizer(raw_input, padding = True, truncation = True, return_tensors = "tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[ 101, 1045, 2031, 2042, 3403, 2005, 2023, 4495, 2026, 2878, 2166,
         999,  102],
       [ 101, 1045, 5223, 2009, 2061, 2172, 1012,  102,    0,    0,    0,
           0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]], dtype=int32)>}


The output itself is a dictionary containing two keys, input_ids and attention_mask. input_ids contains two rows of integers (one for each sentence) that are the unique identifiers of the tokens in each sentence.

## Going through the model

We can download our pretrained model the same way we did with our tokenizer. 🤗 Transformers provides an *TFAutoModel* class which also has a *from_pretrained* method.

In [5]:
from transformers import TFAutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModel.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertModel: ['classifier', 'pre_classifier', 'dropout_19']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


The vector output by the Transformer module is usually large. It generally has three dimensions:

1. Batch size: The number of sequences processed at a time.
2. Sequence length: The length of the numerical representation of the sequence.
3. Hidden size: The vector dimension of each model input.

In [6]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 13, 768)


For our example, we will need a model with a sequence classification head (to be able to classify the sentences as positive or negative). So, we won’t actually use the TFAutoModel class, but TFAutoModelForSequenceClassification.

In [7]:
from transformers import TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(inputs)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_38']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
print(outputs.logits.shape)

(2, 2)


If we look at the shape of our inputs, the dimensionality is much lower: the model head takes as input the high-dimensional vectors we saw before, and outputs vectors containing two values (one per label). Since we have just two sentences and two labels, the result we get from our model is of shape 2 x 2.

## Post-processing

The values we get as output from our model don’t necessarily make sense by themselves. 

In [9]:
print(outputs.logits)

tf.Tensor(
[[-3.2590752  3.3960826]
 [ 4.3236094 -3.4814062]], shape=(2, 2), dtype=float32)


Our model predicted [-3.2590752  3.3960826] for the first sentence and [ 4.3236094 -3.4814062] for the second one. Those are not probabilities but logits, the raw, unnormalized scores outputted by the last layer of the model. To be converted to probabilities, they need to go through a SoftMax layer (all 🤗 Transformers models output the logits, as the loss function for training will generally fuse the last activation function, such as SoftMax, with the actual loss function, such as cross entropy).
SoftMax is often used as the last activation function of a neural network to normalize the output of a network to a probability distribution over predicted output classes.

In [10]:
import tensorflow as tf

pred = tf.math.softmax(outputs.logits, axis = -1)
print(pred)

tf.Tensor(
[[1.2857096e-03 9.9871433e-01]
 [9.9959248e-01 4.0751894e-04]], shape=(2, 2), dtype=float32)


We can see that the model predicted [0.0012, 0.9987] for the first sentence and [0.9995, 0.0004] for the second one. These are recognizable probability scores.

To get the labels corresponding to each position, we can inspect the id2label attribute of the model config.

In [11]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

Now we can conclude that the model predicted the following:

* First sentence: NEGATIVE: 0.0012, POSITIVE: 0.9987
* Second sentence: NEGATIVE: 0.9995, POSITIVE: 0.0004



